# Time Travel Debugger project

## Personal Information

In [1]:
PROJECT_TYPE = 2
NAME = ["Daniel Gusenburger", "Daniel Tabellion"]
ID = ["2544941", "2555742"]
IMPLEMENTED = set()

## High-level Explanation:

We structured our code into `model`, `view` and `domain` folders.

Our debugger starts by running through the given function that should be examined, tracing every execution point.
The `TimeTravelTracer` in the `tracer.py` module is responsible for building up the information we need for later stepping through the given function. Each point of execution is captured by an instance of an `ExecStateDiff`.
Each `ExecStateDiff` stores a list of currently open function scopes(represented by the model class `FunctionStateDiff`) and the action that it performed("CALL","RETURN", "UPDATE" and "EXCEPTION"). These function scopes contain some information about the function itself, added variables and the values of changed variable before and after the update.
The tracer also builds a source map that contains all source lines of functions that we trace.

When done tracing, the list of diffs is given to the `TimeTravelDebugger` in the `debugger.py`.
This class is responsible for taking commands and mapping these to simple step commands implemented in the `StateMachine` as well as for processing breakpoints and watchpoints. 
The `StateMachine` keeps track of the absolute values for variables and class members for open function scopes while stepping through the programm.

With this compartmenalization we make sure, that we can use the main debugger implementation for both the CLI and the GUI without having to rewrite any of the debugging logic.

In [2]:
import sys
!{sys.executable} -m pip install colorama pygments lxml

from time_travel_debugger.view.cli import TimeTravelCLI, next_inputs
from main import remove_html_markup, test1, test2, class_test

Defaulting to user installation because normal site-packages is not writeable


## Must-have Features:

**/R1 `Quit`:**
    
For quitting we just call `sys.exit(1)`.
Also we catch KeyboardInterrupt and EOF exceptions, such that CTRL-C does not leave the debugger, which leads to a cleaner user experience.

One can leave the debugger via the `quit` command or with CTRL-D, which opens a prompt, that asks if you really want to leave.


In [3]:
next_inputs("quit")
with TimeTravelCLI():
    remove_html_markup("<h1>Hello World!</h1>")

IMPLEMENTED.add("R1")

  76  def remove_html_markup(s):
  77>     tag = False
  78      quote = False
  79      out = ""
  80      module(1, 4)
  81  
  82      # Help
  83      # blub
  84  
  85      for c in s:
  86          if c == "<" and not quote:
  87              tag = True
  88          elif c == ">" and not quote:
  89              tag = False
  90          elif c == '"' or c == "'" and tag:
  91              quote = not quote
  92          elif not tag:
  93              out = out + c
  94      return out
Hit start of program
(debugger) quit


**/R2 `Help`:**

For the help command, we used the already present structure from the lecture, so every command, that is named `*_command` is interpreted as a command and appears in the help list. Also docstrings are printed after each command, to give a better understanding of the commands.

In [4]:
next_inputs("help", "quit")
with TimeTravelCLI():
    remove_html_markup("<h1>Hello World!</h1>")

IMPLEMENTED.add("R2")

  76  def remove_html_markup(s):
  77>     tag = False
  78      quote = False
  79      out = ""
  80      module(1, 4)
  81  
  82      # Help
  83      # blub
  84  
  85      for c in s:
  86          if c == "<" and not quote:
  87              tag = True
  88          elif c == ">" and not quote:
  89              tag = False
  90          elif c == '"' or c == "'" and tag:
  91              quote = not quote
  92          elif not tag:
  93              out = out + c
  94      return out
Hit start of program
(debugger) help
backstep        -- Step to the previous instruction
backuntil       -- Execute backward until a given point
break           -- Insert a breakpoint at the given location
breakpoints     -- List all breakpoints
cond            -- Set a conditional breakpoint at the given location
continue        -- Continue execution forward until a breakpoint is hit
delete          -- Remove the given breakpoint
disable         -- Disable the given breakpoint
down            -

**/R3:**

For each command we performed some argument validation in the corresponding `debugger.py` commands.

In [5]:
IMPLEMENTED.add("R3")

**/R4 `Step` and /R5 `Backstep`:**

For stepping forwards (and backwards) we implemented the `StateMachine` class which, as the name implies, represents a simple state machine (We can interpret all possible states of the program to be debugged as a simple state machine, where our stored diffs are translations and the absolute current state of variables in function scopes are the nodes).
The purpose of the state machine is to implement `step` and `backstep`, in order to later build more complex movement commands leveraging these two.

Importantly, for this we need `step` and `backstep` to correctly build(or restore) the absolute state of variables from the given diffs.
In order to make this easier we added the `FunctionStates` helper class, which is a dictionary that maps from function names to a list of open scopes and their local variable values at the current point in time. Also for each function we store, which of these scopes is the active one. This represents the absolute state of any function scope at any given point in time.
The StateMachine always stores its current point in the diff list, so it knows what action comes next.

When calling a function, we append a new scope (dictionary of variables) to the list of open scopes for this function, add the parameters given by the current diff to that dictionary and point at that new scope as active.
Step one diff further.

Reverting a function call deletes the topmost scope of the called function(We don't lose information here, since we can restore its state again, when performing the call while stepping forward).
Go one diff back.

When we return from a function we do nothing but go to the next diff.
We do not delete the newest scope, since we want to keep it in case we step backwards and need to restore it.
We also don't need to update or add any variables, since a return always triggers after we updated the state for the last line of a function.

Reverting a return does nothing but going one diff back, for the same reasons as for return.

When not doing a call or a return, we update the state of the variables, given the next diff.
We add any added variables and update the state of variables that got changed in the current scope of the active function.

Reverting an update removes added variables from the active scope and reverts the values before the update(stored in the diff).

If the current diff's action is an "EXCEPTION" we do nothing, since this is the end of the tracing and printing the exception is part of the CLI, not the debugger class.

By showing that these two actions are sound we can make our lifes a lot easier for all other movement commands!

We also have a `@trigger_update` annotation for each movement function that triggers the UI to update. This ensures, that we always print the current state of the pointer in the code.

In [6]:
next_inputs(*["step"]*9, *["backstep"]*9, "quit")

with TimeTravelCLI():
    test1(10)


IMPLEMENTED.add("R4")
IMPLEMENTED.add("R5")

  13  def test1(a):
  14>     x = 100
  15      id(a)
  16      while a > 5:
  17          # comment
  18          a -= 1
  19      return
Hit start of program
(debugger) step
  13  def test1(a):
  14      x = 100
  15>     id(a)
  16      while a > 5:
  17          # comment
  18          a -= 1
  19      return
(debugger) step
  22  def id(a):
  23>     b = 1234
  24      c = 12
  25      return c
(debugger) step
  22  def id(a):
  23      b = 1234
  24>     c = 12
  25      return c
(debugger) step
  22  def id(a):
  23      b = 1234
  24      c = 12
  25>     return c
(debugger) step
  13  def test1(a):
  14      x = 100
  15>     id(a)
  16      while a > 5:
  17          # comment
  18          a -= 1
  19      return
(debugger) step
  13  def test1(a):
  14      x = 100
  15      id(a)
  16>     while a > 5:
  17          # comment
  18          a -= 1
  19      return
(debugger) step
  13  def test1(a):
  14      x = 100
  15      id(a)
  16      while a > 5:
  17          # co

**/R6 `next` and /R7 `previous`:**

`next` is the same as `until` without parameters.
`previous` is the same as `backuntil` without parameters.
See until/backuntil.

In [7]:
next_inputs(*["next"]*3, *["previous"]*3, "quit")

with TimeTravelCLI():
    test1(10)


IMPLEMENTED.add("R6")
IMPLEMENTED.add("R7")

  13  def test1(a):
  14>     x = 100
  15      id(a)
  16      while a > 5:
  17          # comment
  18          a -= 1
  19      return
Hit start of program
(debugger) next
  13  def test1(a):
  14      x = 100
  15>     id(a)
  16      while a > 5:
  17          # comment
  18          a -= 1
  19      return
(debugger) next
  13  def test1(a):
  14      x = 100
  15      id(a)
  16>     while a > 5:
  17          # comment
  18          a -= 1
  19      return
(debugger) next
  13  def test1(a):
  14      x = 100
  15      id(a)
  16      while a > 5:
  17          # comment
  18>         a -= 1
  19      return
(debugger) previous
  13  def test1(a):
  14      x = 100
  15      id(a)
  16>     while a > 5:
  17          # comment
  18          a -= 1
  19      return
(debugger) previous
  13  def test1(a):
  14      x = 100
  15>     id(a)
  16      while a > 5:
  17          # comment
  18          a -= 1
  19      return
(debugger) previous
  13  def test1(a):
  14>     x = 100

**/R8 `finish`:**

Call `step` until the current diff's stored action is "return" and the current diff is in the same "depth" of execution.
With depth we denote the number of open functions (like in a callstack).
By that we ensure, that we ignore return actions from other functions and we realy only run right before the return action of the current function is performed.

If we perform `finish` on the end of a function this condition obviously holds and we only update the UI.

In [8]:
next_inputs("finish", "quit")

with TimeTravelCLI():
    test1(10)

next_inputs("step", "step", "finish", "quit")
print()
print("Now showing that finish always goes to the end of the current function")
print()
with TimeTravelCLI():
    test1(10)
    
IMPLEMENTED.add("R8")

  13  def test1(a):
  14>     x = 100
  15      id(a)
  16      while a > 5:
  17          # comment
  18          a -= 1
  19      return
Hit start of program
(debugger) finish
  13  def test1(a):
  14      x = 100
  15      id(a)
  16      while a > 5:
  17          # comment
  18          a -= 1
  19>     return
Hit end of program
(debugger) quit

Now showing that finish always goes to the end of the current function

  13  def test1(a):
  14>     x = 100
  15      id(a)
  16      while a > 5:
  17          # comment
  18          a -= 1
  19      return
Hit start of program
(debugger) step
  13  def test1(a):
  14      x = 100
  15>     id(a)
  16      while a > 5:
  17          # comment
  18          a -= 1
  19      return
(debugger) step
  22  def id(a):
  23>     b = 1234
  24      c = 12
  25      return c
(debugger) finish
  22  def id(a):
  23      b = 1234
  24      c = 12
  25>     return c
(debugger) quit


**/R9 `start`:**

Similar as for `finish`, but call `backstep` until the current diff's  action in the same depth is "CALL".

In [9]:
next_inputs("finish", "start", "quit")

with TimeTravelCLI():
    test1(10)

next_inputs("finish", "previous", "previous", "previous", "backstep", "start", "quit")
print()
print("Now showing that start always goes to the start of the current function")
print()
with TimeTravelCLI():
    test1(10)

IMPLEMENTED.add("R9")

  13  def test1(a):
  14>     x = 100
  15      id(a)
  16      while a > 5:
  17          # comment
  18          a -= 1
  19      return
Hit start of program
(debugger) finish
  13  def test1(a):
  14      x = 100
  15      id(a)
  16      while a > 5:
  17          # comment
  18          a -= 1
  19>     return
Hit end of program
(debugger) start
  13  def test1(a):
  14>     x = 100
  15      id(a)
  16      while a > 5:
  17          # comment
  18          a -= 1
  19      return
Hit start of program
(debugger) quit

Now showing that start always goes to the start of the current function

  13  def test1(a):
  14>     x = 100
  15      id(a)
  16      while a > 5:
  17          # comment
  18          a -= 1
  19      return
Hit start of program
(debugger) finish
  13  def test1(a):
  14      x = 100
  15      id(a)
  16      while a > 5:
  17          # comment
  18          a -= 1
  19>     return
Hit end of program
(debugger) previous
  13  def test1(a):
  14      x = 100
  1

**/R10/ `until`:**

For the until command we wrote a argument parser function in the `debugger.py`, that maps the given arguments to just a line number and optionally the filename, which makes writing `until` easier.
This is done via the source map, that we build during parsing.
We can map the current functions name, given by the current diff, to the full source code lines of this function.

**/R100 `until \<line_number\>`:**

We implement this like the other requirements for until, so that we search for the next occurence of the given line (or next executable line, skipping comments etc.), iterating through loops if neccesary (this is fine according to the [forum](https://cms.cispa.saarland/debug/forum/viewtopic.php?f=10&t=44)).
When no line is given act like `next`. So step to next (executable) line and if at the end of a function step out of it.
This is why the `next` command is just a specialization of `until` and we can implement it as `until` without parameters.

So to implement this feature we first determine our target line, we want to step to. When a line number is given, this is the target, if not the target is the current line + 1 (move to the next line). Then we check if this target is actually an executable line, adjusting it to the next possible executable line (For this we essentially loop over the source of the current function and for each line we check if it contains comments etc).
Then when this target is determined, `step` until we either hit the target or we return from a function (stepping out of the function and staying there).

**/R101 `until \<filename\>:\<line_number\>`:**

This works the same as in /R100, but we additionally check whether the target line is hit in a specific file (we store the file name of a given line in the diff as well).

**/R102 `until \<function_name\>`:**

The parser converts function names to the corresponding line number, so this is the same as R/100.

**/R103 `until \<filename\>:\<function_name\>`:**

The parser converts function names to the corresponding line number, so this is the same as R/101.

In [10]:
next_inputs("until 72", "quit")

with TimeTravelCLI():
    test2()
    
IMPLEMENTED.add("R100")

next_inputs("until test_module.py:4", "quit")

print()
print("Until filename:linenumber")
print()

with TimeTravelCLI():
    test2()

IMPLEMENTED.add("R101")

next_inputs("until id", "quit")

print()
print("Until function_name")
print()

with TimeTravelCLI():
    test2()
    
IMPLEMENTED.add("R102")

next_inputs("until test_module.py:module", "quit")

print()
print("Until filename:function_name")
print()

with TimeTravelCLI():
    test2()
    

IMPLEMENTED.add("R103")

  68  def test2():
  69>     a = 5
  70      b = 6
  71      id(a)
  72      module(a, b)
  73      c = 7
Hit start of program
(debugger) until 72
  68  def test2():
  69      a = 5
  70      b = 6
  71      id(a)
  72>     module(a, b)
  73      c = 7
(debugger) quit

Until filename:linenumber

  68  def test2():
  69>     a = 5
  70      b = 6
  71      id(a)
  72      module(a, b)
  73      c = 7
Hit start of program
(debugger) until test_module.py:4
   1  def module(a, b):
   2      d = 8
   3      while d < 10:
   4>         a *= b+d
   5          d += 1
   6      return d
(debugger) quit

Until function_name

  68  def test2():
  69>     a = 5
  70      b = 6
  71      id(a)
  72      module(a, b)
  73      c = 7
Hit start of program
(debugger) until id
  68  def test2():
  69      a = 5
  70      b = 6
  71>     id(a)
  72      module(a, b)
  73      c = 7
(debugger) quit

Until filename:function_name

  68  def test2():
  69>     a = 5
  70      b = 6
  71      id(a)
  72      

**/R110-R113/:**

Analogous to /R100-R103/, but step backwards and check if a function got called.

In [11]:
next_inputs("finish", "backuntil 70", "quit")

with TimeTravelCLI():
    test2()
    
IMPLEMENTED.add("R110")

next_inputs("finish", "backuntil test_module.py:4", "quit")

print()
print("Backuntil filename:linenumber")
print()

with TimeTravelCLI():
    test2()

IMPLEMENTED.add("R111")

next_inputs("finish", "backuntil id", "quit")

print()
print("Backuntil function_name")
print()

with TimeTravelCLI():
    test2()
    
IMPLEMENTED.add("R112")

next_inputs("finish", "backuntil test_module.py:module", "quit")

print()
print("Backuntil filename:function_name")
print()

with TimeTravelCLI():
    test2()
    

IMPLEMENTED.add("R113")

  68  def test2():
  69>     a = 5
  70      b = 6
  71      id(a)
  72      module(a, b)
  73      c = 7
Hit start of program
(debugger) finish
  68  def test2():
  69      a = 5
  70      b = 6
  71      id(a)
  72      module(a, b)
  73>     c = 7
Hit end of program
(debugger) backuntil 70
  68  def test2():
  69      a = 5
  70>     b = 6
  71      id(a)
  72      module(a, b)
  73      c = 7
(debugger) quit

Backuntil filename:linenumber

  68  def test2():
  69>     a = 5
  70      b = 6
  71      id(a)
  72      module(a, b)
  73      c = 7
Hit start of program
(debugger) finish
  68  def test2():
  69      a = 5
  70      b = 6
  71      id(a)
  72      module(a, b)
  73>     c = 7
Hit end of program
(debugger) backuntil test_module.py:4
   1  def module(a, b):
   2      d = 8
   3      while d < 10:
   4>         a *= b+d
   5          d += 1
   6      return d
(debugger) quit

Backuntil function_name

  68  def test2():
  69>     a = 5
  70      b = 6
  71      id(a)
  72    

  73>     c = 7
Hit end of program
(debugger) backuntil test_module.py:module
  68  def test2():
  69      a = 5
  70      b = 6
  71>     id(a)
  72      module(a, b)
  73      c = 7
(debugger) quit


### /R12 Continue and /R13 Reverse

We implemented a function to check wether we arrived at a breakpoint that is called after every `step` or `backstep`. For `continue` and `reverse` we then just simply perform `step`/`backstep` in a loop. Thus we always break if we encounter a breakpoint.

In [12]:
next_inputs("break 85", "continue", "print", "continue", "print" , "finish", "reverse", "print", "reverse", "print", "quit")

with TimeTravelCLI():
    remove_html_markup("<h1>Hello World</h1>")
    
IMPLEMENTED.add("R12")
IMPLEMENTED.add("R13")

  76  def remove_html_markup(s):
  77>     tag = False
  78      quote = False
  79      out = ""
  80      module(1, 4)
  81  
  82      # Help
  83      # blub
  84  
  85      for c in s:
  86          if c == "<" and not quote:
  87              tag = True
  88          elif c == ">" and not quote:
  89              tag = False
  90          elif c == '"' or c == "'" and tag:
  91              quote = not quote
  92          elif not tag:
  93              out = out + c
  94      return out
Hit start of program
(debugger) break 85
Breakpoint 1 added.
(debugger) continue
  76  def remove_html_markup(s):
  77      tag = False
  78      quote = False
  79      out = ""
  80      module(1, 4)
  81  
  82      # Help
  83      # blub
  84  
  85>     for c in s:
  86          if c == "<" and not quote:
  87              tag = True
  88          elif c == ">" and not quote:
  89              tag = False
  90          elif c == '"' or c == "'" and tag:
  91              quote = not quote


(debugger) continue
  76  def remove_html_markup(s):
  77      tag = False
  78      quote = False
  79      out = ""
  80      module(1, 4)
  81  
  82      # Help
  83      # blub
  84  
  85>     for c in s:
  86          if c == "<" and not quote:
  87              tag = True
  88          elif c == ">" and not quote:
  89              tag = False
  90          elif c == '"' or c == "'" and tag:
  91              quote = not quote
  92          elif not tag:
  93              out = out + c
  94      return out
Breakpoint hit!
(debugger) print
s = '<h1>Hello World</h1>'
tag = True
quote = False
out = ''
c = '<'
(debugger) finish
  76  def remove_html_markup(s):
  77      tag = False
  78      quote = False
  79      out = ""
  80      module(1, 4)
  81  
  82      # Help
  83      # blub
  84  
  85#     for c in s:
  86          if c == "<" and not quote:
  87              tag = True
  88          elif c == ">" and not quote:
  89              tag = False
  90          elif c == '"

  79      out = ""
  80      module(1, 4)
  81  
  82      # Help
  83      # blub
  84  
  85>     for c in s:
  86          if c == "<" and not quote:
  87              tag = True
  88          elif c == ">" and not quote:
  89              tag = False
  90          elif c == '"' or c == "'" and tag:
  91              quote = not quote
  92          elif not tag:
  93              out = out + c
  94      return out
Breakpoint hit!
(debugger) print
s = '<h1>Hello World</h1>'
tag = False
quote = False
out = 'Hello World'
c = '>'
(debugger) reverse
  76  def remove_html_markup(s):
  77      tag = False
  78      quote = False
  79      out = ""
  80      module(1, 4)
  81  
  82      # Help
  83      # blub
  84  
  85>     for c in s:
  86          if c == "<" and not quote:
  87              tag = True
  88          elif c == ">" and not quote:
  89              tag = False
  90          elif c == '"' or c == "'" and tag:
  91              quote = not quote
  92          elif not tag:

### Callstack features

In every diff, we track which functions are "open", i.e. we track the callstack depth. We then build the callstack on the fly when `where`, `up` or `down` is called.

In `down`, we push the last position from the current function to a queue, move backwards to the last function called using `until` and then step into this function.

In `up` the last position saved in the queue mentioned in `down` is restored.

In [13]:
next_inputs("until id", "step", "where", "down", "up", "quit")

with TimeTravelCLI():
    test2()

  68  def test2():
  69>     a = 5
  70      b = 6
  71      id(a)
  72      module(a, b)
  73      c = 7
Hit start of program
(debugger) until id
  68  def test2():
  69      a = 5
  70      b = 6
  71>     id(a)
  72      module(a, b)
  73      c = 7
(debugger) step
  22  def id(a):
  23>     b = 1234
  24      c = 12
  25      return c
(debugger) where
#2: id at /home/daniel/Dropbox/Uni/Vorlesungsmaterialien/Automated Debugging/time-travel-debugger/main.py
#1: test2 at /home/daniel/Dropbox/Uni/Vorlesungsmaterialien/Automated Debugging/time-travel-debugger/main.py
(debugger) down
  68  def test2():
  69      a = 5
  70      b = 6
  71>     id(a)
  72      module(a, b)
  73      c = 7
(debugger) up
  22  def id(a):
  23>     b = 1234
  24      c = 12
  25      return c
(debugger) quit


### Inspecting Code and Variables

For the `print` command we mainly used the implementation that was provided in the Debugging Book. We modified it a bit to exclude the `__builtins__` variable that contains python specific built-in variables and functions such as `__file__` and `__name__` and we are not interested in that.

In [14]:
next_inputs("step", "step", "step", "print", "print a+10", "quit")

with TimeTravelCLI():
    class_test(1, 2)
    
IMPLEMENTED.add("R171")
IMPLEMENTED.add("R171")
IMPLEMENTED.add("R18")

  55  def class_test(a, b):
  56>     test = Test(a, b)
  57      return test
Hit start of program
(debugger) step
  42      def __init__(self, a, b):
  43>         self.x = a + b
  44          self.y = b - b
  45          return
(debugger) step
  42      def __init__(self, a, b):
  43          self.x = a + b
  44>         self.y = b - b
  45          return
(debugger) step
  42      def __init__(self, a, b):
  43          self.x = a + b
  44          self.y = b - b
  45>         return
(debugger) print
self = {'x': 3, 'y': 0}
a = 1
b = 2
(debugger) print a+10
a+10 = 11
(debugger) quit


For the `list` command we adapted the implementation from the Debugging Book to use our source map which we built during the tracing step. That way we can easily find the source of any function that was executed and display it.

In the `list` command we used `pygments` `TerminalFormatter` to get a coloured output.

In [15]:
next_inputs("list", "step", "step", "step", "list 5", "list 8 2", "list module", "list foo", "quit")

with TimeTravelCLI():
    remove_html_markup("<h1>Hello World</h1>")
    
IMPLEMENTED.add("R161")
IMPLEMENTED.add("R162")
IMPLEMENTED.add("R163")

  76  def remove_html_markup(s):
  77>     tag = False
  78      quote = False
  79      out = ""
  80      module(1, 4)
  81  
  82      # Help
  83      # blub
  84  
  85      for c in s:
  86          if c == "<" and not quote:
  87              tag = True
  88          elif c == ">" and not quote:
  89              tag = False
  90          elif c == '"' or c == "'" and tag:
  91              quote = not quote
  92          elif not tag:
  93              out = out + c
  94      return out
Hit start of program
(debugger) list
  76  def remove_html_markup(s):
  77>     tag = False
  78      quote = False
  79      out = ""
  80      module(1, 4)
  81  
  82      # Help
  83      # blub
  84  
  85      for c in s:
  86          if c == "<" and not quote:
  87              tag = True
  88          elif c == ">" and not quote:
  89              tag = False
  90          elif c == '"' or c == "'" and tag:
  91              quote = not quote
  92          elif not tag:
  93            

(debugger) quit


### Watchpoints (variable + expression)

For watchpoints we implemented a simple container class that keeps track of the expression, its current value and its last observed value as well as the id. We can then update the current value by storing the current value in the last value, and storing the result from the evaluation of the expression in the current context.

This makes it very easy to watch expressions as well as variables, since we use the same mechanism for both.

We can then easily check after each movement command whether a watchpoint has changed by comparing the last and current value for every watchpoint.

We actively decided not to check for duplicate watch (and also breakpoints) since this is not done in modern IDEs such as PyCharm either.

In [16]:
next_inputs("watch a", "watch 2 * a - 5", "watch", "break 16", *["continue"]*3, "unwatch 1", *["continue"]*2, "quit")

with TimeTravelCLI():
    test1(10)
    
IMPLEMENTED.add("R190")
IMPLEMENTED.add("R191")

  13  def test1(a):
  14>     x = 100
  15      id(a)
  16      while a > 5:
  17          # comment
  18          a -= 1
  19      return
Hit start of program
(debugger) watch a
Added watchpoint with id 1.
(debugger) watch 2 * a - 5
Added watchpoint with id 2.
(debugger) watch
      id       | watched expression |     value     
----------------------------------------------------
       1       |         a          |      10       
       2       |     2 * a - 5      |      15       
(debugger) break 16
Breakpoint 1 added.
(debugger) continue
  13  def test1(a):
  14      x = 100
  15      id(a)
  16>     while a > 5:
  17          # comment
  18          a -= 1
  19      return
Breakpoint hit!
(debugger) continue
  13  def test1(a):
  14      x = 100
  15      id(a)
  16>     while a > 5:
  17          # comment
  18          a -= 1
  19      return
'a': 10 -> 9
'2 * a - 5': 15 -> 13
Breakpoint hit!
(debugger) continue
  13  def test1(a):
  14      x = 100
  15      id(a)
  16>     

### Breakpoints (Line, Function, Conditional)

To implement line, function and conditional breakpoints, we first implemented a base class that contains basic information about breakpoints, i.e. information that is needed for every Breakpoint such as which file it is in, whether it is active or not, its id and what type (such that we don't have to infer it using `instanceof`)

In [17]:
from time_travel_debugger.model.breakpoint import Breakpoint, FunctionBreakpoint, BaseBreakpoint

print([x for x in BaseBreakpoint.__dict__.keys() if not x.startswith('__')])

['toggle', 'enable', 'disable', 'id', 'filename', 'abs_filename', 'active', 'status', 'breakpoint_type']


Furthermore, we implemented some __magic methods__ such as `__iter__` for easier usage in the rest of the program and methods like `enable` and `disable` that are used to set the breakpoint's status. 
We always store the absolute filename and use this for comparison, since there is a possibility (although very small) that there are two modules with the same name but different path.

After implementing the base class, we subclassed it and created a `Breakpoint` class for line and conditional breakpoints as well as a `FunctionBreakpoint` class for -- as the name already says -- function breakpoints.

The main difference between those is that the regular `Breakpoint` only stores its location in the form of a single line number whereas the `FunctionBreakpoint` stores -- in addition to the function name -- two linenumbers. One for the first line of the function definition and one for the last one. We can then check whether we have to break by checking whether currently are at the correct position.

In [18]:
next_inputs("break 70", "break test_module.py:module", "break id", "list", "continue", "continue", "continue", "continue", "quit")

with TimeTravelCLI():
    test2()

IMPLEMENTED.add("R201")
IMPLEMENTED.add("R202")
IMPLEMENTED.add("R203")

  68  def test2():
  69>     a = 5
  70      b = 6
  71      id(a)
  72      module(a, b)
  73      c = 7
Hit start of program
(debugger) break 70
Breakpoint 1 added.
(debugger) break test_module.py:module
Breakpoint 2 added.
(debugger) break id
Breakpoint 3 added.
(debugger) list
  68  def test2():
  69>     a = 5
  70#     b = 6
  71      id(a)
  72      module(a, b)
  73      c = 7
(debugger) continue
  68  def test2():
  69      a = 5
  70>     b = 6
  71      id(a)
  72      module(a, b)
  73      c = 7
Breakpoint hit!
(debugger) continue
  22  def id(a):
  23>     b = 1234
  24      c = 12
  25      return c
Breakpoint hit!
(debugger) continue
  68  def test2():
  69      a = 5
  70#     b = 6
  71      id(a)
  72      module(a, b)
  73>     c = 7
Hit end of program
(debugger) continue
  68  def test2():
  69      a = 5
  70#     b = 6
  71      id(a)
  72      module(a, b)
  73>     c = 7
Hit end of program
(debugger) quit


In [19]:
next_inputs("break 70", "break test_module.py:module", "break id", "breakpoints", "quit")

with TimeTravelCLI():
    test2()
    
IMPLEMENTED.add("R204")

  68  def test2():
  69>     a = 5
  70      b = 6
  71      id(a)
  72      module(a, b)
  73      c = 7
Hit start of program
(debugger) break 70
Breakpoint 1 added.
(debugger) break test_module.py:module
Breakpoint 2 added.
(debugger) break id
Breakpoint 3 added.
(debugger) breakpoints
      id       | type |        location         |    active     |     condition      
-------------------------------------------------------------------------------------
       1       | line |       main.py:70        |    active     |                    
       2       | func |  test_module.py:module  |    active     |                    
       3       | func |       main.py:id        |    active     |                    
(debugger) quit


In [20]:
next_inputs("break 70", "breakpoints", "delete 1", "breakpoints", "quit")

with TimeTravelCLI():
    test2()
    
IMPLEMENTED.add("R205")

  68  def test2():
  69>     a = 5
  70      b = 6
  71      id(a)
  72      module(a, b)
  73      c = 7
Hit start of program
(debugger) break 70
Breakpoint 1 added.
(debugger) breakpoints
      id       | type |        location         |    active     |     condition      
-------------------------------------------------------------------------------------
       1       | line |       main.py:70        |    active     |                    
(debugger) delete 1
(debugger) breakpoints
      id       | type |        location         |    active     |     condition      
-------------------------------------------------------------------------------------
(debugger) quit


In [21]:
next_inputs("break 70", "breakpoints", "disable 1", "breakpoints", "list", "continue", "start", "enable 1", "breakpoints", "continue", "quit")

with TimeTravelCLI():
    test2()
    
IMPLEMENTED.add("R206")
IMPLEMENTED.add("R207")

  68  def test2():
  69>     a = 5
  70      b = 6
  71      id(a)
  72      module(a, b)
  73      c = 7
Hit start of program
(debugger) break 70
Breakpoint 1 added.
(debugger) breakpoints
      id       | type |        location         |    active     |     condition      
-------------------------------------------------------------------------------------
       1       | line |       main.py:70        |    active     |                    
(debugger) disable 1
(debugger) breakpoints
      id       | type |        location         |    active     |     condition      
-------------------------------------------------------------------------------------
       1       | line |       main.py:70        |   inactive    |                    
(debugger) list
  68  def test2():
  69>     a = 5
  70      b = 6
  71      id(a)
  72      module(a, b)
  73      c = 7
(debugger) continue
  68  def test2():
  69      a = 5
  70      b = 6
  71      id(a)
  72      module(a, b)
  73>     c = 7
Hi

In [22]:
next_inputs("cond 16 a > 7", *["continue", "print"]*4, "quit")

with TimeTravelCLI():
    test1(10)
    
IMPLEMENTED.add("R208")

  13  def test1(a):
  14>     x = 100
  15      id(a)
  16      while a > 5:
  17          # comment
  18          a -= 1
  19      return
Hit start of program
(debugger) cond 16 a > 7
(debugger) continue
  13  def test1(a):
  14      x = 100
  15      id(a)
  16>     while a > 5:
  17          # comment
  18          a -= 1
  19      return
Breakpoint hit!
(debugger) print
a = 10
x = 100
(debugger) continue
  13  def test1(a):
  14      x = 100
  15      id(a)
  16>     while a > 5:
  17          # comment
  18          a -= 1
  19      return
Breakpoint hit!
(debugger) print
a = 9
x = 100
(debugger) continue
  13  def test1(a):
  14      x = 100
  15      id(a)
  16>     while a > 5:
  17          # comment
  18          a -= 1
  19      return
Breakpoint hit!
(debugger) print
a = 8
x = 100
(debugger) continue
  13  def test1(a):
  14      x = 100
  15      id(a)
  16#     while a > 5:
  17          # comment
  18          a -= 1
  19>     return
Hit end of program
(debugger) print
a

In [23]:
print("\n".join(f"{i}: {x}" for i, x in enumerate(list(sorted(IMPLEMENTED, key=lambda x: int(x[1:]))), start=1)))

1: R1
2: R2
3: R3
4: R4
5: R5
6: R6
7: R7
8: R8
9: R9
10: R12
11: R13
12: R18
13: R100
14: R101
15: R102
16: R103
17: R110
18: R111
19: R112
20: R113
21: R161
22: R162
23: R163
24: R171
25: R190
26: R191
27: R201
28: R202
29: R203
30: R204
31: R205
32: R206
33: R207
34: R208


## May-have Features:

**Storing differences and Call Stack:**

We only store the difference for each point in execution, not the absolute state after the execution.
Also we do not store the callstack for each diff, but rather build the callstack during stepping from knowledge we have from the diffs(function name, file name and current line for each diff).
This minimizes the storage used for the diffs.

### Syntax highlighting in the CLI

We used `pygments` to generate syntax highlighting in the CLI. (see previous demonstrations)

### Autocompletion

We used the `readline` module to use arrow keys and Ctrl-P/Ctrl-N to navigate through the history of entered commands. Additionally, we used `readline.set_completer` and `readline.parse_and_bind` to autocomplete the currently entered command on tab press or alternatively print the commands that are possible completions of the current input. 
To find all matches, we use a simple prefix matching as demonstrated in the class below:

In [24]:
import inspect
from time_travel_debugger.view.completer import CLICompleter

print(inspect.getsource(CLICompleter))

class CLICompleter(object):

    def __init__(self, options):
        self._options = options

    def complete(self, text, state):
        if state == 0:
            if not text:
                self._matches = self._options[:]

            else:
                self._matches = [
                    s for s in self._options if s.startswith(text)]
        try:
            return self._matches[state]
        except IndexError:
            return None



## GUI Must-haves:

We implemented a GUI in Jupyter Notebook with a mix of mostly Jupyter Notebook Widgets and some plain HTML.

All the main features of the CLI are implemented.

### Movement Commands

One can do following movement commands:`step`, `next`, `finish`, `continue`, each with a dedicated button.
Their corresponding counter parts for walking back in time exist as well.

### Breakpoints

### Search

We implemented a query-based search function, with which it is possible to find events for **variable changes(var )**, **function calls(call )** and **breakpoint hits(hit )** in the program execution.
For each of these three event types, there are different search criteria. This is the syntax:

    - "[event]  <string>"      :   Events with id/variable name/function name, depending on the event type
    - "[event] -func <string>" :   All events in the given function  
    - "[event] -line <int>"    :   All events in the given linet  

It is possible to refine the search for events by multiple search criteria.       
Examples:

    - Search for all variable changes in line 89                  : "var -line 89"
    - Search for all function calls in line 89                    : "call -line 89"
    - Search for all hits of breakpoint \#1 in the function 'foo' : "hit 1 -func foo

""

# GUI

We decided to go for the simple approach of building our application using Jupyter Notebook widgets. However it turned out it was not as flexible as we thought it would be, so we had to resort to using a bunch of buttons for what could have been done with a simple responsive layout, e.g. inserting and deleting breakpoints by clicking on the respective lines.

Same thing for watchpoints.

Logic-wise we could use the same debugger due to how we designed the infrastructure that we improved by adding a search feature as already stated in the section above.
We could also use most of the commands from the CLI with little adaption. We wanted to encapsulate these core commands in a base class that could be used in both cases, but we ran out of time, so we had to use duplicate implementations.

## Code display

Initially, we used the `list_command` from the cli, which turned out ok, but the scrolling did not behave as we wanted, so we made the switch to using HTML instead.

The syntax highlighting was again done using [`pygments`](https://pygments.org/docs/formatters/). This time however, we used the `HtmlFormatter` to generate highlighted HTML. The downside here was that we could not easily highlight the current line and the breakpoints. We solved this issue by parsing the generated HTML, which contained `span`-tags with an id for each line allowing us to select all tags with a CSS selector. To parse and query HTML we used the open source [`lxml`](https://lxml.de/lxmlhtml.html) module.

We iterate over all available breakpoints and add a `class="active|inactive"` to the corresponding tag, depending on the state of the breakpoint. We do a similar thing for the currentline. Then, we inject custom CSS wrapped in a `<style>` tag using Jupyter's `HTML` display which allows us to style the `code` pane however we want.

## Navigation commands

Nothing special here, we added buttons for each movement command we wanted to display and linked it to the corresponding command. The command implementation are practically the same as in the CLI. We decided to leave out the `until` and `backuntil` command for usability reasons, since we could not e.g. click on a line to move there. So the commands we mapped to the GUI are `step`, `backstep`, `next`, `previous`, `start` and `finish`.

Furthermore, we added a slider that allows us to move freely through the execution using Jupyter notebook's `IntSlider` widget.

By `jsdlink`ing a `Play` widget to this slider we can also automatically step through the execution. With another slider, this time linked to the `Play` widget's `interval` trait, we can also control how fast we want to step though the recorder execution. Since building the diffs and updating is quite performance intensive, we limited it to a delay of maximum 100ms. However, we did not time the execution so we can not confirm that we actually get 100ms, so it might be slower.

Lastly, we added a `Toggle Button` to switch between forward and backward playback.

## Variables and Watchpoints

We used simple Markdown tables in combination with `Output` widgets to display the current local variables as well as all watchpoints.
To add/remove watchpoints we included two buttons, a `Text` widget to enter the expression we want to watch and a `Dropdown` widget containing all available watchpoints to select one that we want to remove.

## Breakpoints

To manage breakpoints, we added a second tab since we did not want to overload the code display. In this second tab, we added buttons to remove, disable, and add new breakpoints. To enter the arguments that are processed in the `add_breakpoint` button, we first added a `Dropdown` to select the breakpoint type. We furthermore added a `Dropdown` to select a function from the sourcemap and two `Text` widgets for the line number and condition. Depending on the breakpoint type, we disable the unneeded input widgets.

The available breakpoints are displayed again using a `Markdown` table. With another `Dropdown`, the user can select a breakpoint to modify using the `disable` and `delete` buttons.

## Search

Last but not least, we added a third tab for the search. In the search tab we simply display a single text area where we can input the same search strings as in the CLI (see above). The results are displayed in a simple `Output` widget. Next to each search result is a button that moves the execution to the point in time where we recorder this change.

In [25]:
from time_travel_debugger.view.gui import GUI

with GUI():
    remove_html_markup("<h1>Hello World</h1>")

HTML(value='\n            <style>\n               .jupyter-widgets-output-area .output_scroll {\n             …